In [1]:
import requests
import json
import csv
import pandas as pd
from itertools import chain
from collections import Counter
import ast
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
# Load the CSV. 

file_path = 'movies_data.csv'  # Replace with the actual file name or path
movie_data_df = pd.read_csv(file_path)

pd.set_option('display.max_colwidth', 13)
display(movie_data_df)

,title,release_date,original_language,runtime,genres,actors,directors,production_companies,keywords,popularity,vote_count,vote_average,budget,revenue
0,Four Rooms,1995-12-09,en,98,[{'id': 8...,['Tim Rot...,['Allison...,[{'id': 1...,"['hotel',...",21.376,2357,5.758,4000000,4257354
1,Judgment ...,1993-10-15,en,109,[{'id': 2...,['Emilio ...,['Stephen...,[{'id': 1...,['chicago...,7.675,284,6.556,21000000,12136938
2,Star Wars,1977-05-25,en,121,[{'id': 1...,['Mark Ha...,['George ...,[{'id': 1...,['android...,120.016,18563,8.208,11000000,775398007
3,Finding Nemo,2003-05-30,en,100,[{'id': 1...,['Albert ...,['Andrew ...,[{'id': 3...,['parent ...,74.931,17613,7.823,94000000,940335536
4,Forrest Gump,1994-06-23,en,142,[{'id': 3...,['Tom Han...,['Robert ...,[{'id': 4...,['vietnam...,52.131,24570,8.481,55000000,677387716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,Jackass P...,2013-10-22,en,92,[{'id': 3...,['Johnny ...,['Jeff Tr...,[{'id': 4...,['duringc...,14.690,1407,6.104,15000000,151831537
6996,Forbidden...,2014-01-30,ru,128,[{'id': 5...,['Jason F...,['Oleg St...,[{'id': 7...,['monster...,9.949,252,5.900,26000000,34592118
6997,Batman v ...,2016-03-23,en,152,[{'id': 2...,['Ben Aff...,['Zack Sn...,[{'id': 1...,['vigilan...,65.151,16721,5.900,250000000,873637528
6998,Zombie Br...,1986-05-22,en,92,[{'id': 2...,[],['Carmelo...,[],"['gore', ...",1.129,7,2.500,10,15


In [3]:
# CLEANING THE DATA SET 


# Add column for net profit
movie_data_df['net_profit'] = movie_data_df['revenue'] - movie_data_df['budget']
# Add column for profitability yes or no. Create new column 'profitable'
movie_data_df['profitable'] = (movie_data_df['revenue'] > movie_data_df['budget']).astype(int)


# Change budget, revenue and net profit to result in (millions).
movie_data_df['budget'] = movie_data_df['budget'] / 1000000
movie_data_df['revenue'] = movie_data_df['revenue'] / 1000000
movie_data_df['net_profit'] = movie_data_df['net_profit'] / 1000000
movie_data_df = movie_data_df.rename(columns={'budget': 'budget ($M)'})
movie_data_df = movie_data_df.rename(columns={'revenue': 'revenue ($M)'})
movie_data_df = movie_data_df.rename(columns={'net_profit': 'net_profit ($M)'})

# Change column names
movie_data_df = movie_data_df.rename(columns={'release_date' : 'year'})
movie_data_df = movie_data_df.rename(columns={'runtime' : 'runtime_minutes'})
movie_data_df = movie_data_df.rename(columns={'production_companies' : 'studios'})
movie_data_df = movie_data_df.rename(columns={'popularity' : 'TMDB_popularity'})
movie_data_df = movie_data_df.rename(columns={'vote_average' : 'vote_rating'})

In [4]:
# Add a column for budget brackets. 

# Define a function to categorize the budget
def categorize_budget(budget):
    if budget < 5.0:
        return 'low'
    elif 5 <= budget <= 50:
        return 'medium'
    else:
        return 'high'

# Add a new 'budget_bracket' column to the DataFrame
movie_data_df['budget_bracket'] = movie_data_df['budget ($M)'].apply(categorize_budget)

# Reorder the columns to place 'budget_bracket' next to 'budget' using index positions
column_names = movie_data_df.columns.tolist()
column_order = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 13, 14, 15]
reordered_columns = [column_names[i] for i in column_order]
movie_data_df = movie_data_df[reordered_columns]


In [5]:
# Cleaning - MODIFY GENRES, STUDIOS, ACTORS AND DIRECTORS 

from ast import literal_eval

# Define the functions to extract genre and studio names
def extract_genre_names(genre_list):
    return [genre['name'] for genre in genre_list]

def extract_studio_names(studio_list):
    return [studio['name'] for studio in studio_list]


# Convert the strings to lists of dictionaries using literal_eval
movie_data_df['genres'] = movie_data_df['genres'].apply(literal_eval)
movie_data_df['studios'] = movie_data_df['studios'].apply(literal_eval)

# Apply the functions to the appropriate columns
movie_data_df['genres'] = movie_data_df['genres'].apply(extract_genre_names)
movie_data_df['studios'] = movie_data_df['studios'].apply(extract_studio_names)



















display(movie_data_df)

,title,year,original_language,runtime_minutes,genres,actors,directors,studios,keywords,TMDB_popularity,vote_count,vote_rating,budget ($M),budget_bracket,revenue ($M),net_profit ($M),profitable
0,Four Rooms,1995-12-09,en,98,"[Crime, C...",['Tim Rot...,['Allison...,"[Miramax,...","['hotel',...",21.376,2357,5.758,4.00000,low,4.257354,0.257354,1
1,Judgment ...,1993-10-15,en,109,"[Action, ...",['Emilio ...,['Stephen...,[Largo En...,['chicago...,7.675,284,6.556,21.00000,medium,12.136938,-8.863062,0
2,Star Wars,1977-05-25,en,121,[Adventur...,['Mark Ha...,['George ...,[Lucasfil...,['android...,120.016,18563,8.208,11.00000,medium,775.398007,764.398007,1
3,Finding Nemo,2003-05-30,en,100,[Animatio...,['Albert ...,['Andrew ...,[Pixar],['parent ...,74.931,17613,7.823,94.00000,high,940.335536,846.335536,1
4,Forrest Gump,1994-06-23,en,142,"[Comedy, ...",['Tom Han...,['Robert ...,[Paramoun...,['vietnam...,52.131,24570,8.481,55.00000,high,677.387716,622.387716,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,Jackass P...,2013-10-22,en,92,[Comedy],['Johnny ...,['Jeff Tr...,[Paramoun...,['duringc...,14.690,1407,6.104,15.00000,medium,151.831537,136.831537,1
6996,Forbidden...,2014-01-30,ru,128,[Thriller...,['Jason F...,['Oleg St...,[Russian ...,['monster...,9.949,252,5.900,26.00000,medium,34.592118,8.592118,1
6997,Batman v ...,2016-03-23,en,152,"[Action, ...",['Ben Aff...,['Zack Sn...,[Warner B...,['vigilan...,65.151,16721,5.900,250.00000,high,873.637528,623.637528,1
6998,Zombie Br...,1986-05-22,en,92,"[Action, ...",[],['Carmelo...,[],"['gore', ...",1.129,7,2.500,0.00001,low,0.000015,0.000005,1


In [6]:
################## CLEANING DONE
################## NOW PERFORM ENCODING 
##################
##################
##################
##################
# For genres I am one-hot-encoding.
# For studios I am doing N most frequent words
# For keywords I am N most frequent words. 
# For actors and directors N-most frequent encoded 

In [7]:
# CONVERT COLUMNS TO PROPER FORMAT FOR ONE-HOT-ENCODING

#  Datframe is named movies_data_df
first_two_rows = movie_data_df.head(2)
dict_representation = first_two_rows.to_dict(orient='list')

print(dict_representation)



{'title': ['Four Rooms', 'Judgment Night'], 'year': ['1995-12-09', '1993-10-15'], 'original_language': ['en', 'en'], 'runtime_minutes': [98, 109], 'genres': [['Crime', 'Comedy'], ['Action', 'Crime', 'Thriller']], 'actors': ["['Tim Roth', 'Jennifer Beals', 'Antonio Banderas', 'Valeria Golino', 'David Proval', 'Sammi Davis', 'Amanda de Cadenet', 'Madonna', 'Ione Skye', 'Lili Taylor', 'Alicia Witt', 'Tamlyn Tomita', 'Lana McKissack', 'Danny Verduzco', 'Patricia Vonne', 'Salma Hayek', 'Quentin Tarantino', 'Paul Calderon', 'Bruce Willis', 'Marisa Tomei', 'Kathy Griffin', 'Lawrence Bender', 'Kimberly Blair', 'Marc Lawrence', 'Paul Skemp']", "['Emilio Estevez', 'Cuba Gooding Jr.', 'Denis Leary', 'Stephen Dorff', 'Jeremy Piven', 'Peter Greene', 'Michael DeLorenzo', 'Everlast', 'Michael Wiseman', 'Relioues Webb', 'Eugene Williams', 'Christine Harnos', 'Galyn Görg', 'Angela Alvarado', 'Will Zahrn']"], 'directors': ["['Allison Anders', 'Alexandre Rockwell', 'Robert Rodriguez', 'Quentin Tarantino'

In [8]:
def clean_column(cell):
    if isinstance(cell, str):
        # Convert the string representation of a list to an actual list
        cell = ast.literal_eval(cell)
    return [str(item).strip().strip("'") for item in cell]

columns_to_clean = ['genres', 'actors', 'directors', 'studios', 'keywords']

for column in columns_to_clean:
    movie_data_df[column] = movie_data_df[column].apply(clean_column)

In [9]:
# Display the updated DataFrame
pd.set_option('display.max_colwidth', 20)
display(movie_data_df)
movie_data_df.head

,title,year,original_language,runtime_minutes,genres,actors,directors,studios,keywords,TMDB_popularity,vote_count,vote_rating,budget ($M),budget_bracket,revenue ($M),net_profit ($M),profitable
0,Four Rooms,1995-12-09,en,98,"[Crime, Comedy]","[Tim Roth, Jenni...","[Allison Anders,...","[Miramax, A Band...","[hotel, new year...",21.376,2357,5.758,4.00000,low,4.257354,0.257354,1
1,Judgment Night,1993-10-15,en,109,"[Action, Crime, ...","[Emilio Estevez,...",[Stephen Hopkins],[Largo Entertain...,"[chicago, illino...",7.675,284,6.556,21.00000,medium,12.136938,-8.863062,0
2,Star Wars,1977-05-25,en,121,"[Adventure, Acti...","[Mark Hamill, Ha...",[George Lucas],"[Lucasfilm Ltd.,...","[android, galaxy...",120.016,18563,8.208,11.00000,medium,775.398007,764.398007,1
3,Finding Nemo,2003-05-30,en,100,"[Animation, Family]","[Albert Brooks, ...",[Andrew Stanton],[Pixar],[parent child re...,74.931,17613,7.823,94.00000,high,940.335536,846.335536,1
4,Forrest Gump,1994-06-23,en,142,"[Comedy, Drama, ...","[Tom Hanks, Robi...",[Robert Zemeckis],"[Paramount, The ...",[vietnam veteran...,52.131,24570,8.481,55.00000,high,677.387716,622.387716,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,Jackass Presents...,2013-10-22,en,92,[Comedy],[Johnny Knoxvill...,[Jeff Tremaine],"[Paramount, MTV ...",[duringcreditsst...,14.690,1407,6.104,15.00000,medium,151.831537,136.831537,1
6996,Forbidden Empire,2014-01-30,ru,128,"[Thriller, Adven...","[Jason Flemyng, ...",[Oleg Stepchenko],[Russian Film Gr...,"[monster, mystic...",9.949,252,5.900,26.00000,medium,34.592118,8.592118,1
6997,Batman v Superma...,2016-03-23,en,152,"[Action, Adventu...","[Ben Affleck, He...",[Zack Snyder],[Warner Bros. Pi...,"[vigilante, supe...",65.151,16721,5.900,250.00000,high,873.637528,623.637528,1
6998,Zombie Brigade,1986-05-22,en,92,"[Action, Horror,...",[],"[Carmelo Musca, ...",[],"[gore, zombie]",1.129,7,2.500,0.00001,low,0.000015,0.000005,1


<bound method NDFrame.head of                     title        year original_language  runtime_minutes  \
0              Four Rooms  1995-12-09                en               98   
1          Judgment Night  1993-10-15                en              109   
2               Star Wars  1977-05-25                en              121   
3            Finding Nemo  2003-05-30                en              100   
4            Forrest Gump  1994-06-23                en              142   
...                   ...         ...               ...              ...   
6995  Jackass Presents...  2013-10-22                en               92   
6996     Forbidden Empire  2014-01-30                ru              128   
6997  Batman v Superma...  2016-03-23                en              152   
6998       Zombie Brigade  1986-05-22                en               92   
6999  The Grand Seduction  2014-05-30                en              112   

                   genres               actors           

In [10]:
unique_counts = {}

for column in ['genres', 'actors', 'directors', 'studios', 'keywords']:
    unique_counts[column] = movie_data_df[column].explode().nunique()

print(unique_counts)


{'genres': 19, 'actors': 121289, 'directors': 3426, 'studios': 6923, 'keywords': 12313}


In [11]:
# Here I'm encoding the 50 most frequent categories for actors, directors, studios and keywords. 

# This is because {'genres': 19, 'actors': 121289, 'directors': 3426, 'studios': 6923, 'keywords': 12313}

# So encoding normally would make the dataset far too dimensional. 

def top_n_categories(df, column, n):
    top_n = df[column].explode().value_counts().nlargest(n).index.tolist()
    return top_n

def filter_top_n(df, column, top_n):
    df[column] = df[column].apply(lambda x: [item for item in x if item in top_n])
    return df

# This is N-most frequent categories number 

n = 100
n2 = 120

for column in ['actors', 'directors', 'studios']:
    top_n_categories_list = top_n_categories(movie_data_df, column, n)
    movie_data_df = filter_top_n(movie_data_df, column, top_n_categories_list)
    
    
for column in ['keywords']:
    top_n_categories_list = top_n_categories(movie_data_df, column, n2)
    movie_data_df = filter_top_n(movie_data_df, column, top_n_categories_list)

# Now, movie_data_df contains only the top N-most frequent categories for each specified column

In [12]:
# Encoding

def one_hot_encode(df, column):
    mlb = MultiLabelBinarizer()
    encoded = mlb.fit_transform(df[column])
    encoded_df = pd.DataFrame(encoded, columns=[f'{column}_{category}' for category in mlb.classes_])
    return encoded_df

# Perform one-hot encoding on the filtered columns
actors_encoded = one_hot_encode(movie_data_df, 'actors')
directors_encoded = one_hot_encode(movie_data_df, 'directors')
studios_encoded = one_hot_encode(movie_data_df, 'studios')
keywords_encoded = one_hot_encode(movie_data_df, 'keywords')
genres_encoded = one_hot_encode(movie_data_df, 'genres')

# Concatenate the encoded DataFrames with the original DataFrame
movie_data_transformed = pd.concat([movie_data_df, genres_encoded, actors_encoded, directors_encoded, studios_encoded, keywords_encoded], axis=1)

# Drop the original columns
movie_data_transformed = movie_data_transformed.drop(columns=['genres', 'actors', 'directors', 'studios', 'keywords'])


In [13]:
# Final step is to remove any film where net profitability = 0, because they seem to all be faulty data.
# This changes it to 6942 movies. 

movie_data_filtered = movie_data_transformed[movie_data_transformed['net_profit ($M)'] != 0]


In [14]:
####### IMPORTANT STEP, REPLACE SPACES WITH _

movie_data_filtered.columns = movie_data_filtered.columns.str.replace(" ", "_")


In [15]:
# Save the DataFrame to a CSV file
movie_data_filtered.to_csv('movies_data_encoded.csv', index=False, encoding='utf-8')


In [16]:
############################# FINISHED

############################ MOVIES_DATA_ENCODED READY FOR ANALYSIS IN R 